In [17]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [18]:
def write_json_to_txt_raw(json_data, file_name):
    with open(file_name, 'w') as file:
        for test_case, details in json_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [19]:
def write_json_to_txt(json_data, file_name):
    filtered_data = {k: v for k, v in json_data.items() if v.get("reflect") == "yes"}
    with open(file_name, 'w') as file:
        for test_case, details in filtered_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [20]:
TEST_CASE_GENERATOR_PROMPT="""
I want you to act as a software tester.
Your task is to read the test scenario's name and the corresponding use case specification to base on those information for generateing test steps for test cases and their following expected result.
Return the test cases in json format.
The JSON format should follow the following structure:
{
  "Test Case 1":[
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "explanation": "Why do you create this test case? How does this test case related to the test scenario inputed?",
  ],
}
If there are more than one test case for this scenario, continue writing other test case in this form.

Rules for generating test steps:
- Describe the test step clearly to make sure each test case is independent, tester do not need to read other information (example: other test case, use case specification) to know how to do that step.
- Avoid references to other test cases or instructions like "do as mentioned".
- If the test case need to be repeated to test with different order, data or case, seperate them to be distinct test cases.
- If the scenario is about testing the displation and there is no flow directly cover that scenario, use only the basic (or main) flow to test it.
- If there are use cases mentioned in extended or included use case, create test case combine use cases, Try to find the connection point of use cases for combination. 
- For test scenarios mentioning navigation in the name, only produce test cases related to the specified navigation method. 
(Example: "Scenario: User navigates to a page by navbar", only produce a test case of user navigates to that page by navbar even though the use case description has many way to navigate to that page)
- For test scenarios not mentioning navigation in the name, do not include any navigation test cases.
- Generate test cases that directly match the scenario name. Choose only one flow to cover the scenario.
- Ensure all actions and objectives match the scenario name.
"""

In [21]:
TEST_CASE_VALIDATOR="""
Given a test scenario and test cases to test that given test scenario.
Mark if test case can test the given test scenario or not through test steps, expected output, objective (although if it test other use case path or flow, if it is not used to test the given test scenario.)
And give explanation why you think the resulted test case reflect the given test scenario or not. 
Your response should keep the format of the inserted test cases.
The JSON format should follow the following structure:
{
  "Test Case 1":[
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "reflect": "yes/no",
    "explanation": "explain why you think this test case reflect the given test scenario or not",
  ],
}"""

In [22]:
scenario =  """New Password Does Not Meet Requirements"""

In [23]:
usecase_content = """
Use case Name
[User + Shop] Change password
Brief description
This use case provides a field for users to change their password.
Actors
Customers (Users and shops)
Basic Flow
Customers who want to change their password should input all the information in 3 fields given, namely “Old Password”, “New Password”, “Retype New Password”.
Once customers have completely confirmed the new password, they can hit the save button to save the new password.
Alternative Flows
Alternative flow 1: Customer does not remember password
From #1 of the basic flow, the customer cannot proceed with the basic flow, the customer needs to contact the administrator with their username and the reason for their request for further assistance and will have to wait until the administrator resets their password.
Alternative flow 2: Customer types in the old password
The customer follows the steps in #2 of the basic flow and receives a confirmation message, but the database does not reflect the customer’s input.
Pre-conditions
User/ shop has already signed in. They can edit the password by filling a form in their profile page. 
Post-conditions
Password of the account must be updated if the customer’s new password and re-entered new password are the same.
"""

In [24]:

promptTestCaseGenerator = [
{ "role": "system", "content": TEST_CASE_GENERATOR_PROMPT},
{ "role": "user", "content": scenario + "\n" + usecase_content}
]
gpt_response = ask(promptTestCaseGenerator, client, model)
json_data = json.loads(gpt_response)
print(gpt_response)
promptTestCaseValidate = [
    { "role": "system", "content": TEST_CASE_VALIDATOR},
    { "role": "user", "content": scenario + "\n" + gpt_response}
]
final_response = ask(promptTestCaseValidate, client, model)
json_fin = json.loads(final_response)
print(final_response)

{
  "Test Case 1": {
    "testCaseName": "Change Password with New Password Not Meeting Requirements",
    "objective": "Verify that the system prevents the password change when the new password does not meet the specified requirements",
    "testSteps": [
      "Step 1: Sign in with a valid user/shop account.",
      "Step 2: Navigate to the profile page.",
      "Step 3: Click on the 'Change Password' option.",
      "Step 4: Enter the current password in the 'Old Password' field.",
      "Step 5: Enter a new password that does not meet the specified requirements in the 'New Password' field.",
      "Step 6: Re-enter the new password in the 'Retype New Password' field.",
      "Step 7: Click on the 'Save' button."
    ],
    "expectedResult": "The system should display an error message indicating that the new password does not meet the requirements and the password should not be changed.",
    "explanation": "This test case is created to ensure that the system enforces password requi